In [9]:
%pip install --upgrade --quiet pymilvus[model] langchain-milvus langchain-openai
%pip install -qU langchain_community pypdf

zsh:1: no matches found: pymilvus[model]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_ollama import OllamaLLM

In [11]:
CONNECTION_URI = "http://localhost:19530"

In [12]:
ollama_embedding_func = OllamaLLM(model="moondream")

In [13]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Ollama Setup
def setup_ollama_pdf_vectorstore(file_path, embedding_model="moondream"):
    try:
        # 1. Load PDF
        loader = PyMuPDFLoader(file_path)
        docs = loader.load()

        # 2. Text Splitting (Optional but recommended)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        splits = text_splitter.split_documents(docs)

        # 3. Ollama Embeddings
        embeddings = OllamaEmbeddings(
            model=embedding_model,  # You can change this to other Ollama models
            num_gpu=16  # Optional: specify GPU usage
        )

        # 4. Create Chroma Vector Store
        vector_store = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            collection_name="ollama_pdf_collection"
        )

        return vector_store

    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None

files = os.listdir("papers")
print(files)

vector_store = setup_ollama_pdf_vectorstore(f"papers/{files[0]}")
print(f"Vector store for {files[0]} created successfully")
print("")

['2303.05510v1.pdf']
Vector store for 2303.05510v1.pdf created successfully



In [14]:
# Perform Similarity Search
contexts = ""
query = input("Enter your query: ")

if vector_store:
    results = vector_store.similarity_search(query, k=10)
    
    for doc in results:
        print("\n--- Relevant Document Excerpt ---")
        print(doc.page_content)
        contexts = contexts + doc.page_content


--- Relevant Document Excerpt ---
Published as a conference paper at ICLR 2023
APPENDIX
In this appendix, we supplement the main paper by providing more thorough empirical evaluations to
back up our claims and more detailed descriptions of the algorithms to help readers better understand
our paper.
This appendix is organized as follows.
• In Sec. A, we provide more comprehensive results of our algorithm and the baseline algorithms.
We also include the license information of the datasets we use.
• In Sec. B, we consider the scenario where test cases are not provided. We evaluate our PG-TD
algorithm using automatically-generated test cases.
• In Sec. C, we provide empirical evidence for our claims in the discussion section that our
algorithm is versatile and can be used to optimize different code generation objectives other
than the pass rate. We consider the objectives of generating concise codes and generating
codes with more comments.

--- Relevant Document Excerpt ---
Published as a

In [15]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize Ollama Chat Model
chat_model = ChatOllama(
    model="llama3.2:1b",  # You can change to other models like mistral, phi3
    temperature=0.7,  # Creativity level
    num_gpu=16  # Optional GPU configuration
)

# Simple Chat Interaction
messages = [
    SystemMessage(content="You are a helpful assistant for my research."),
    HumanMessage(content= f"Query + {query} with the context + {contexts}")
]

# Generate Response
response = chat_model.invoke(messages)
print("--- Ollama Response ---")
print(response.content)

--- Ollama Response ---
The text describes a software tool for generating code solutions to specific problems in an interview setting, called "prophet". Here's a breakdown of the key points:

**Problem Statement**

Prophet aims to generate solutions to problems from standard input (integer values) using a combination of natural language and mathematical reasoning. The goal is to produce solutions that meet certain constraints, such as passing certain tests or having an integer output.

**Input/Output**

* Input: Integer values on standard input
* Output: String representations of generated code solutions with specific formatting

**Features**

* Optimized for performance using various techniques (e.g., beam search)
* Can generate solutions from problem inputs in multiple formats (e.g., pass rate, strict accuracy)

**Methodology**

1. **Problem Definition**: Problem description and constraints are provided.
2. **Input Processing**: Input values are processed to ensure they meet the spec